Name: Shirui Chen
ID: 6958613091

In [1]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import os
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
def encode_text_tfidf_vectors(in_folder='/train/', subject=False, parameter=1):
    os.chdir(in_folder)
    files = os.listdir(in_folder)
    x = []
    y = []
    tmp = []

    for file in files:
        path = os.path.join(in_folder, file)
        with open(path, 'r',errors='ignore') as f:
            t = f.readlines()
            if subject == 'True':
                tmp.append(t[0])
            else:
                tmp.append(t[2])
    vectorizer = TfidfVectorizer(min_df = parameter)
    x = vectorizer.fit_transform(tmp)
    for filename in files:
        if 'legit' in filename:
            y.append(0)
        elif 'spmsg' in filename:
            y.append(1)
    return x,y

In [3]:
x,y=encode_text_tfidf_vectors('/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW3/files/train')

In [4]:
x = x.toarray()
x = pd.DataFrame(x)
x

,0,1,2,3,4,5,6,7,8,9,...,4736,4737,4738,4739,4740,4741,4742,4743,4744,4745
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
4,0.0,0.009181,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.0,0.047950,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
67,0.0,0.006157,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.006531,0.0,0.007695
68,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.022436,0.0,...,0.024613,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
69,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000


In [5]:
y = pd.DataFrame(y)
y.columns = ['spam']
y

,spam
0,1
1,0
2,1
3,0
4,0
...,...
66,0
67,0
68,0
69,0


In [6]:
def train_models(x_train,y_train,x_test,y_test,model):
    global clf
    if model == 'decision_tree':
        clf = tree.DecisionTreeClassifier()
    elif model == 'naive_bayes':
        clf = BernoulliNB()
    elif model == 'linear_classifier':
        clf = make_pipeline(StandardScaler(with_mean=False), SGDClassifier(loss='squared_loss'))
    elif model == 'gradient_boosting_classifier':
        clf = GradientBoostingClassifier()
    elif model == 'k-nearest_neighbors':
        clf = KNeighborsClassifier()
    scores = cross_val_score(clf, x, y, cv=5).mean()
    return scores

In [7]:
res = pd.DataFrame(index=range(3),columns=range(5))
res.columns = ['decision_tree','naive_bayes','linear_classifier','gradient_boosting_classifier','k-nearest_neighbors']

In [8]:
e1 = encode_text_tfidf_vectors(in_folder='/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW3/files/train/', subject=False, parameter=0.9)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
dt1 = train_models(x_train,y_train,x_test,y_test,'decision_tree')
nb1 = train_models(x_train,y_train,x_test,y_test,'naive_bayes')
lc1 = train_models(x_train,y_train,x_test,y_test,'linear_classifier')
gbc1 = train_models(x_train,y_train,x_test,y_test,'gradient_boosting_classifier')
knn1 = train_models(x_train,y_train,x_test,y_test,'k-nearest_neighbors')
res.iloc[0] = [dt1,nb1,lc1, gbc1, knn1]

In [9]:
e2 = encode_text_tfidf_vectors(in_folder='/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW3/files/train/', subject=False, parameter = 0.8)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
dt2 = train_models(x_train,y_train,x_test,y_test,'decision_tree')
nb2 = train_models(x_train,y_train,x_test,y_test,'naive_bayes')
lc2 = train_models(x_train,y_train,x_test,y_test,'linear_classifier')
gbc2 = train_models(x_train,y_train,x_test,y_test,'gradient_boosting_classifier')
knn2 = train_models(x_train,y_train,x_test,y_test,'k-nearest_neighbors')
res.iloc[1] = [dt2,nb2,lc2, gbc2, knn2]

In [10]:
e3 = encode_text_tfidf_vectors(in_folder='/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW3/files/train/', subject=False, parameter = 0.7)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
dt3 = train_models(x_train,y_train,x_test,y_test,'decision_tree')
nb3 = train_models(x_train,y_train,x_test,y_test,'naive_bayes')
lc3 = train_models(x_train,y_train,x_test,y_test,'linear_classifier')
gbc3 = train_models(x_train,y_train,x_test,y_test,'gradient_boosting_classifier')
knn3 = train_models(x_train,y_train,x_test,y_test,'k-nearest_neighbors')
res.iloc[2] = [dt3,nb3,lc3, gbc3, knn3]
print(res)

  decision_tree naive_bayes linear_classifier gradient_boosting_classifier  \
0       0.84381    0.774286              0.44                     0.886667   
1      0.858095    0.774286          0.619048                     0.886667   
2      0.886667    0.774286          0.395238                     0.886667   

  k-nearest_neighbors  
0            0.802857  
1            0.802857  
2            0.802857  


In [11]:
res['decision_tree'].mean()

0.8628571428571429

In [12]:
res['naive_bayes'].mean()

0.7742857142857144

In [13]:
res['linear_classifier'].mean()

0.4847619047619047

In [14]:
res['gradient_boosting_classifier'].mean()

0.8866666666666667

In [15]:
res['k-nearest_neighbors'].mean()

0.8028571428571428

In [16]:
x,y = encode_text_tfidf_vectors(in_folder='/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW3/files/train/')
dc = tree.DecisionTreeClassifier()
final_dc = dc.fit(x, y)
gbc = GradientBoostingClassifier()
final_gbc = gbc.fit(x,y)

In [17]:
x_test,y_test = encode_text_tfidf_vectors(in_folder='/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW3/files/test/')
x_test = pd.DataFrame(x_test.toarray())
tmp = pd.DataFrame(columns=range(len(x.toarray()[0])-len(x_test.columns)))
df1 = pd.concat([x_test,tmp],axis=1)
df1 = df1.fillna(0)
dc_acc = final_dc.score(df1,y_test)
gbc_acc = final_gbc.score(df1,y_test)
print(dc_acc,gbc_acc)

0.7183098591549296 0.7464788732394366


The accuracy of decision tree is 0.7183, and the accuracy of gradient boosting is 0.7465.
The difference is smaller compared to performance difference on cross-validation.

In [18]:
len(y_train)

56

In [19]:
print(y_train[y_train.spam==1].sum())

spam    10
dtype: int64


In [20]:
random.seed(1)
arr = np.zeros(56).reshape(-1,1)
arr[:10] = 1
np.random.shuffle(arr)
arr

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.]])

In [21]:
clf = BernoulliNB()
final_model = clf.fit (x_train, arr)
scores = cross_val_score(clf, x_train, arr, cv=5).mean()
scores

0.7863636363636364

In [22]:
clf = KNeighborsClassifier()
final_model = clf.fit (x_train, arr)
scores = cross_val_score(clf, x_train, arr, cv=5).mean()
scores

0.8393939393939395

In [25]:
x,y = encode_text_tfidf_vectors(in_folder='/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW3/files/train/',subject=True)
dc = tree.DecisionTreeClassifier()
final_dc = dc.fit(x, y)
gbc = GradientBoostingClassifier()
final_gbc = gbc.fit(x,y)

In [26]:
x_test,y_test = encode_text_tfidf_vectors(in_folder='/Users/shiruichen/Desktop/2022 Fall/ISE 540/HW3/files/test/',subject=True)
x_test = pd.DataFrame(x_test.toarray())
tmp = pd.DataFrame(columns=range(len(x.toarray()[0])-len(x_test.columns)))
df1 = pd.concat([x_test,tmp],axis=1)
df1 = df1.fillna(0)
dc_acc = final_dc.score(df1,y_test)
gbc_acc = final_gbc.score(df1,y_test)
print(dc_acc,gbc_acc)

0.7323943661971831 0.7464788732394366


The performance doesn't improve.